## Simple Value Investing Dashboard, using python 

This application , refer [Vincent Tatan's blog and project at GitHub ](https://towardsdatascience.com/value-investing-dashboard-with-python-beautiful-soup-and-dash-python-43002f6a97ca) , collect company's stock price history , financial summary information from professional financial website ( MarketWatch) and yahoo ( pandas' datareader) to do simple rule-based analysis and infer reasonable share price then give user a hint about sell or buy decision .

Value investment and technical analysis are two mainstream in stock market. To increase probability of winning , investors now should know more than that in this rapid change era . Maybe choose good funds to buy is better idea for general investors.  Even professional fund managers face big challenge to fast collecting and interpreting information for making good trading decision and quick enough before market change to another condition . Information system could help practitioners do their job easier. 

Python is hot and popular in data science community. Many experts contribute lots of data processing and visualization packages to Python, make it is easy to use and learn if there are good examples. This application show how Python can process web resources and organize information with friendly tabular and chart web page offering a easy to use investment tool.  I have learned a lot through reading codes , developing new function and refactoring part of UI and code to another style . Thanks Vincent share his idea and project. 

Following are some changes I make from that project :
- Change UI layout and apply Bootstrap 4
- Could execute in Jupyter notebook
- Change share price graph to K chart , add SMA , MACD , RSI 
- File name are also changed and do some code refactoring
- Get data through API from stock market data company is more reliable and efficient than parsing web page from website , but this part left no modification.


It's fun and very useful to develop investment portfolio management or stock investment tools. The book ["The Man Who Solved the Market"](https://www.amazon.com/Man-Who-Solved-Market-Revolution/dp/073521798X)  tell the story about how the company , initial core members are lots of top PHD mathematicians , develop information system to beat financial market . The idea of Big Data and AI are already implemented to solve complicate and dynamic changing problem in that era . The book doesn't uncover technical detail, it's business secret to build competition barrier and reduce the impact of similar market strategy. Many company already build similar investment strategy and tool. Since Big Data and AI technology quickly spread to software community and industry. More and more people and company know how to use that technology. Such system may soon be from unique superiority become basic ability of industries.  


.

## Warning : ##
This application has not been rigorously tested and its domain rules are very simple which are hard to cover the complexity of real stock market. There may be also wrong information introduced by some not found bugs.
## <span style='color:#a00000'>This application is not mature enough and risky to be your decision tool in real stock market !</span>



-------


Main process :

1. Get stock ticker and name to be data source of dropdown control
2. Get stock price history from yahoo through pandas datareader when user choose a stock from dropdown control
3. Plot stock k chart , sma  , macd , rsi lines
4. Get stock financial information from financial website to do basic analysis and infer future value for trading decision.


You can refer [Vincent Tatan's blog ](https://towardsdatascience.com/value-investing-dashboard-with-python-beautiful-soup-and-dash-python-43002f6a97ca) to learn complete explanation.

In [3]:
import numpy as np
import numpy_financial as npf
import dash_bootstrap_components as dbc
from dateutil.relativedelta import relativedelta
import webbrowser
import dashboard 
from dashboard import *
import importlib
import logging
from flask.logging import default_handler
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
#%matplotlib inline
cf.offline.go_offline() # with execute this command will cause QuantFig.iplot() run into error

In [4]:
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dashboard_layout()
app.title = 'Simple Value Investing Dashboard'
register_callbacks(app)
if __name__ == '__main__':
    app.run_server(
        debug=True,
        mode='external',    # inline  external  jupyterlab
        port=8123,
        dev_tools_ui=False,
        dev_tools_silence_routes_logging=True) 
    # set mode='external' to use web browser
    # dev_tools_silence_routes_logging=False
    url = 'http://localhost:8123/'
    webbrowser.open(url)
    


Dash app running on http://127.0.0.1:8123/
